In [1]:
import sqlite3
import random
import networkx as nx
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import time



conn = sqlite3.connect("db_numprop-4_nestlim-100.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM data")
all_rows = cursor.fetchall()
column_names = [desc[0] for desc in cursor.description]
formula_idx = column_names.index("formula")
category_idx = column_names.index("category")

In [7]:
device = torch.device("cpu")
print(f"Using device: {device}")
from helpers import (find_allowable_combinations, round_prediction,
                     binary_to_bitlist, tree_to_formula, run_derivation_for_row, check_tree_matches, percent_longer)
from collections import defaultdict

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)


class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.ln1 = nn.Linear(input_size, 16)
        self.ln2 = nn.Linear(16, 16)
        self.ln3 = nn.Linear(16, output_size)

    def forward(self, x):
        x = 2 * x - 1
        x = x.to(device)
        x = F.relu(self.ln1(x))
        x = F.relu(self.ln2(x))
        x = self.ln3(x)
        return torch.sigmoid(x)


MODEL_CACHE = {}
def get_model(input_size, output_size):
    if output_size not in MODEL_CACHE:
        model = Net(input_size, output_size).to(device)
        MODEL_CACHE[output_size] = model
    model = MODEL_CACHE[output_size]
    model.apply(init_weights)  
    return model

COMBINATION_CACHE = {}
def compute_target(correct, input_row, nn_first_prediction, tree_candidate):
    """
    Returns (best_target_tensor, non_terminal_count) or (None, 0).
    Caches the candidate list + count so we don't recompute it every call.
    """
    assignments = tuple(input_row)
    
    # Create a hashable key for the cache - use tuple representation of tree instead of id
    # This is more reliable if trees are recreated but structurally identical
    key = (tuple(str(tree_candidate).encode()), correct, assignments)
    
    # fetch or compute the list of fillings + count
    if key not in COMBINATION_CACHE:
        candidate_list, non_term_count = find_allowable_combinations(
            tree_candidate, correct, assignments
        )
        COMBINATION_CACHE[key] = (candidate_list, non_term_count)
    else:
        candidate_list, non_term_count = COMBINATION_CACHE[key]
    
    # Early return if no valid candidates or non-terminals
    if not candidate_list or non_term_count == 0:
        return None, 0
    
    # Optimize tensor creation for large candidate lists
    if len(candidate_list) > 1000:
        # Process in batches to reduce memory pressure
        batch_size = 1000
        min_dist = float('inf')
        best_vector = None
        
        for i in range(0, len(candidate_list), batch_size):
            batch = candidate_list[i:i+batch_size]
            
            # Create tensor in one operation for efficiency
            batch_data = torch.zeros((len(batch), non_term_count), dtype=torch.float, device=device)
            
            for j, cand in enumerate(batch):
                for var_id, value in cand.items():
                    if var_id.startswith("Z_"):
                        idx = int(var_id[2:])
                        if idx < non_term_count:
                            batch_data[j, idx] = value
            
            # Calculate distances in one operation
            pred = nn_first_prediction.view(1, -1).to(device)
            batch_dists = torch.norm(batch_data - pred, dim=1)
            
            # Update best if found
            batch_min_idx = torch.argmin(batch_dists).item()
            batch_min_dist = batch_dists[batch_min_idx].item()
            
            if batch_min_dist < min_dist:
                min_dist = batch_min_dist
                best_vector = batch_data[batch_min_idx]
        
        return best_vector, non_term_count
    
    else:
        # For smaller lists, use the original approach
        vectors = []
        for cand in candidate_list:
            vec = torch.tensor(
                [cand.get(f"Z_{i}", 0) for i in range(non_term_count)],
                dtype=torch.float,
                device=device
            )
            vectors.append(vec)
        
        tensor_stack = torch.stack(vectors)               # [#candidates, non_term_count]
        # pick the one closest to the network's prediction
        pred = nn_first_prediction.view(1, -1).to(device) # [1, non_term_count]
        dists = torch.norm(tensor_stack - pred, dim=1)    # [#candidates]
        best = torch.argmin(dists).item()
        
        return vectors[best], non_term_count



def train_on_truth_table(nn_model, truth_table, bitlist, tree_candidate,
                         max_epochs=5000, lr=0.001):
    """
    Trains nn_model on the entire truth_table as a single batch.
    Returns:
      - int  : epochs to converge
      - None : if some row is unsolvable
      - inf  : if it doesn't converge within max_epochs
    """
    optimizer = torch.optim.Adam(nn_model.parameters(), lr=lr)
    X = torch.from_numpy(truth_table.astype(np.float32)).to(device)  # [N, in_size]
    N = X.size(0)

    # ——— STEP 1: compute ALL targets up front, and grab output_size ———
    targets = []
    output_size = None
    with torch.no_grad():
        for i in range(N):
            x_row = X[i:i+1]                        # [1, in_size]
            pred = nn_model(x_row).squeeze(0)       # [out_size]
            tgt, nt_count = compute_target(
                bitlist[i], tuple(truth_table[i]), pred, tree_candidate
            )
            if tgt is None:
                return None  # unsolvable
            # set output_size once, from the first row
            if output_size is None:
                output_size = nt_count
            # enforce consistency
            if tgt.numel() != output_size:
                raise ValueError(
                    f"Row {i} target length {tgt.numel()} != expected {output_size}"
                )
            targets.append(tgt)

    # stack into [N, output_size]
    Y = torch.stack(targets)
    assert Y.shape == (N, output_size)

    # ——— STEP 2: batch‐train until convergence or max_epochs ———
    for epoch in range(1, max_epochs+1):
        nn_model.train()
        P = nn_model(X)  # [N, output_size]

        # check if rounding already matches
        if torch.equal(round_prediction(P), round_prediction(Y)):
            return epoch

        loss = F.binary_cross_entropy(P, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return float('inf')


def evaluate_candidate_options(current_options, truth_table, final_formula, bitlist):
    candidate_iterations = {}

    for idx, tree_cand in current_options.items():
        cand_str = tree_to_formula(tree_cand)

        # 1) Exact match to your known minimal formula?
        if cand_str == final_formula:
            return cand_str

        # 2) Fully instantiated (no Z_i’s) → test on every row
        if 'Z' not in cand_str:
            all_correct = True
            for i, row in enumerate(truth_table):
                if not check_tree_matches(tree_cand, bitlist[i], tuple(row)):
                    all_correct = False
                    break

            if all_correct:
                return cand_str
            else:
                continue

        # 3) Otherwise has nonterminals → try fillings via NN
        first_assignment = tuple(truth_table[0])
        candidate_list, non_terminal_count = find_allowable_combinations(
            tree_cand, bitlist[0], first_assignment
        )
        if non_terminal_count == 0 or not candidate_list:
            continue

        nn_model = get_model(truth_table.shape[1], non_terminal_count)
        iters = train_on_truth_table(nn_model, truth_table, bitlist, tree_cand)
        if iters is None or iters == float('inf'):
            continue

        candidate_iterations[idx] = iters

    return candidate_iterations

if __name__ == "__main__":
    unfound_rows = []

    start_time = time.time()
    
    truth_table = np.array([
        [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 0, 0],
        [1, 0, 1, 1], [1, 0, 1, 0], [1, 0, 0, 1], [1, 0, 0, 0],
        [0, 1, 1, 1], [0, 1, 1, 0], [0, 1, 0, 1], [0, 1, 0, 0],
        [0, 0, 1, 1], [0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0]
    ])

    found_count = 0
    minimal_count = 0
    percent_diffs = []           
    found_formulas = []

    # Process all rows with progress tracking
    row_indices = random.sample(range(len(all_rows)), 10)
    for row_index in range(10):
        print(f"Processing row {row_index}...")
        row = all_rows[row_index]

        row = all_rows[row_index]
        final_formula = row[formula_idx]
        bitlist = binary_to_bitlist(row[category_idx], len(truth_table))

        current = None
        found = False    
        
        for depth in range(20):
            COMBINATION_CACHE.clear()
            print(f"  Depth {depth}")
            current_options = run_derivation_for_row(row_index, row, column_names, current)
            result = evaluate_candidate_options(current_options, truth_table, final_formula, bitlist)
        
            if isinstance(result, str):
                found = True       # <-- Mark found!
                found_count += 1
                found_formula = result
                minimal = final_formula
        
                if found_formula == minimal:
                    minimal_count += 1
                    pct = 0.0
                else:
                    pct = percent_longer(found_formula, minimal)
        
                percent_diffs.append(pct)
                found_formulas.append((row_index, found_formula, pct))
                print(f"  Found formula: {found_formula}  ({pct:.1f}% longer)")
                break
            elif result:
                best_key = min(result, key=result.get)
                current = current_options[best_key]
                print(f"  Best candidate: {tree_to_formula(current)} (iterations: {result[best_key]})")
            else:
                print("  No viable candidates found")
                break
        
        # <-- outside the for depth loop
        if not found:
            unfound_rows.append((row_index, final_formula))




    print(f"\n=== Summary ===")
    print(f"  Rows with *any* formula found : {found_count} out of {1000}")
    print(f"  Of those, exact minimals     : {minimal_count}")
    if percent_diffs:
        avg_pct = sum(percent_diffs) / len(percent_diffs)
        print(f"  Average %‐overlength: {avg_pct:.1f}%")

    for idx, formula, pct in found_formulas:
        sign = '+' if pct >= 0 else ''
        print(f" • row {idx}: {formula!r} → {sign}{pct:.1f}%")

    

    # Summarize % overlength by minimal formula length
    length_groups = defaultdict(list)
    
    for (row_index, found_formula, pct) in found_formulas:
        minimal = all_rows[row_index][formula_idx]
        minimal_length = len(minimal)
        length_groups[minimal_length].append(pct)
    
    print("\n=== Overlength Summary by Minimal Formula Length ===")
    for length in sorted(length_groups):
        diffs = length_groups[length]
        avg_pct = sum(diffs) / len(diffs) if diffs else 0
        print(f"  Minimal length {length}: {len(diffs)} formulas, average %‐overlength = {avg_pct:.1f}%")

    if unfound_rows:
        print(f"\n=== Rows where no formula was found ({len(unfound_rows)}) ===")
    for idx, minimal_formula in unfound_rows:
        print(f" • row {idx}: minimal formula was {minimal_formula!r}")


    elapsed = time.time() - start_time
    print(f"Total execution time: {elapsed:.2f} seconds.")

Using device: cpu
Processing row 0...
  Depth 0
Using row 0: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65280, 0, 'p')

Current expression: Z
  Found formula: p  (0.0% longer)
Processing row 1...
  Depth 0
Using row 1: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61680, 0, 'q')

Current expression: Z
  Found formula: q  (0.0% longer)
Processing row 2...
  Depth 0
Using row 2: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52428, 0, 'r')

Current expression: Z
  Found formula: r  (0.0% longer)
Processing row 3...
  Depth 0
Using row 3: (0, 0, 0, 0, 0, 0, 1, 0, 0, 43690, 0, 's')

Current expression: Z
  Found formula: s  (0.0% longer)
Processing row 4...
  Depth 0
Using row 4: (0, 0, 0, 0, 0, 0, 1, 0, 0, 255, 1, 'NA(p,p)')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 142)
  Depth 1
Using row 4: (0, 0, 0, 0, 0, 0, 1, 0, 0, 255, 1, 'NA(p,p)')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(p,Z) (iterations: 97)
  Depth 2
Using row 4: (0, 0, 0, 0, 0, 0, 1, 0, 0, 255, 1, 'NA(p,p)')

Current expression: ('NA', 'p',

  Best candidate: NA(p,Z) (iterations: 11)
  Depth 2
Using row 19: (0, 0, 0, 0, 0, 0, 1, 0, 0, 41215, 2, 'NA(p,NA(q,s))')

Current expression: ('NA', 'p', 'Z')
  Best candidate: NA(p,NA(Z,Z)) (iterations: 13)
  Depth 3
Using row 19: (0, 0, 0, 0, 0, 0, 1, 0, 0, 41215, 2, 'NA(p,NA(q,s))')

Current expression: ('NA', 'p', ('NA', 'Z', 'Z'))
  Best candidate: NA(p,NA(s,Z)) (iterations: 6)
  Depth 4
Using row 19: (0, 0, 0, 0, 0, 0, 1, 0, 0, 41215, 2, 'NA(p,NA(q,s))')

Current expression: ('NA', 'p', ('NA', 's', 'Z'))
  Found formula: NA(p,NA(s,q))  (0.0% longer)
Processing row 20...
  Depth 0
Using row 20: (0, 0, 0, 0, 0, 0, 1, 0, 0, 35071, 2, 'NA(p,NA(r,s))')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 25)
  Depth 1
Using row 20: (0, 0, 0, 0, 0, 0, 1, 0, 0, 35071, 2, 'NA(p,NA(r,s))')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(p,Z) (iterations: 8)
  Depth 2
Using row 20: (0, 0, 0, 0, 0, 0, 1, 0, 0, 35071, 2, 'NA(p,NA(r,s))')

Current expression: ('NA', 'p', 

  Best candidate: NA(r,NA(p,Z)) (iterations: 5)
  Depth 4
Using row 29: (0, 0, 0, 0, 0, 0, 1, 0, 0, 47923, 2, 'NA(r,NA(p,s))')

Current expression: ('NA', 'r', ('NA', 'p', 'Z'))
  Found formula: NA(r,NA(p,s))  (0.0% longer)
Processing row 30...
  Depth 0
Using row 30: (0, 0, 0, 0, 0, 0, 1, 0, 0, 62451, 2, 'NA(r,NA(q,q))')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 22)
  Depth 1
Using row 30: (0, 0, 0, 0, 0, 0, 1, 0, 0, 62451, 2, 'NA(r,NA(q,q))')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(NA(Z,Z),Z) (iterations: 1)
  Depth 2
Using row 30: (0, 0, 0, 0, 0, 0, 1, 0, 0, 62451, 2, 'NA(r,NA(q,q))')

Current expression: ('NA', ('NA', 'Z', 'Z'), 'Z')
  Best candidate: NA(NA(r,Z),Z) (iterations: 19)
  Depth 3
Using row 30: (0, 0, 0, 0, 0, 0, 1, 0, 0, 62451, 2, 'NA(r,NA(q,q))')

Current expression: ('NA', ('NA', 'r', 'Z'), 'Z')
  Best candidate: NA(NA(r,q),Z) (iterations: 13)
  Depth 4
Using row 30: (0, 0, 0, 0, 0, 0, 1, 0, 0, 62451, 2, 'NA(r,NA(q,q))')

Current

  Best candidate: NA(q,NA(q,NA(Z,Z))) (iterations: 15)
  Depth 5
Using row 39: (0, 0, 0, 0, 0, 0, 1, 0, 0, 16383, 3, 'NA(p,NA(p,NA(q,r)))')

Current expression: ('NA', 'q', ('NA', 'q', ('NA', 'Z', 'Z')))
  Best candidate: NA(q,NA(q,NA(r,Z))) (iterations: 3)
  Depth 6
Using row 39: (0, 0, 0, 0, 0, 0, 1, 0, 0, 16383, 3, 'NA(p,NA(p,NA(q,r)))')

Current expression: ('NA', 'q', ('NA', 'q', ('NA', 'r', 'Z')))
  Found formula: NA(q,NA(q,NA(r,p)))  (0.0% longer)
Processing row 40...
  Depth 0
Using row 40: (0, 0, 0, 0, 0, 0, 1, 0, 0, 24575, 3, 'NA(p,NA(p,NA(q,s)))')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 17)
  Depth 1
Using row 40: (0, 0, 0, 0, 0, 0, 1, 0, 0, 24575, 3, 'NA(p,NA(p,NA(q,s)))')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(q,Z) (iterations: 7)
  Depth 2
Using row 40: (0, 0, 0, 0, 0, 0, 1, 0, 0, 24575, 3, 'NA(p,NA(p,NA(q,s)))')

Current expression: ('NA', 'q', 'Z')
  Best candidate: NA(q,NA(Z,Z)) (iterations: 6)
  Depth 3
Using row 40: (0, 0, 0,

  Best candidate: NA(NA(p,NA(p,NA(s,NA(p,s)))),NA(NA(p,Z),Z)) (iterations: 10)
  Depth 13
Using row 43: (0, 0, 0, 0, 0, 0, 1, 0, 0, 20735, 3, 'NA(p,NA(q,NA(p,s)))')

Current expression: ('NA', ('NA', 'p', ('NA', 'p', ('NA', 's', ('NA', 'p', 's')))), ('NA', ('NA', 'p', 'Z'), 'Z'))
  Best candidate: NA(NA(p,NA(p,NA(s,NA(p,s)))),NA(NA(p,s),Z)) (iterations: 1)
  Depth 14
Using row 43: (0, 0, 0, 0, 0, 0, 1, 0, 0, 20735, 3, 'NA(p,NA(q,NA(p,s)))')

Current expression: ('NA', ('NA', 'p', ('NA', 'p', ('NA', 's', ('NA', 'p', 's')))), ('NA', ('NA', 'p', 's'), 'Z'))
  Best candidate: NA(NA(p,NA(p,NA(s,NA(p,s)))),NA(NA(p,s),NA(Z,Z))) (iterations: 13)
  Depth 15
Using row 43: (0, 0, 0, 0, 0, 0, 1, 0, 0, 20735, 3, 'NA(p,NA(q,NA(p,s)))')

Current expression: ('NA', ('NA', 'p', ('NA', 'p', ('NA', 's', ('NA', 'p', 's')))), ('NA', ('NA', 'p', 's'), ('NA', 'Z', 'Z')))
  Best candidate: NA(NA(p,NA(p,NA(s,NA(p,s)))),NA(NA(p,s),NA(p,Z))) (iterations: 3)
  Depth 16
Using row 43: (0, 0, 0, 0, 0, 0, 1, 0, 0, 20

...

  Best candidate: NA(NA(q,s),NA(NA(s,q),NA(NA(s,Z),Z))) (iterations: 10)
  Depth 11
Using row 84: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65450, 3, 'NA(NA(p,p),NA(s,s))')

Current expression: ('NA', ('NA', 'q', 's'), ('NA', ('NA', 's', 'q'), ('NA', ('NA', 's', 'Z'), 'Z')))
  Best candidate: NA(NA(q,s),NA(NA(s,q),NA(NA(s,s),Z))) (iterations: 4)
  Depth 12
Using row 84: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65450, 3, 'NA(NA(p,p),NA(s,s))')

Current expression: ('NA', ('NA', 'q', 's'), ('NA', ('NA', 's', 'q'), ('NA', ('NA', 's', 's'), 'Z')))
  Best candidate: NA(NA(q,s),NA(NA(s,q),NA(NA(s,s),NA(Z,Z)))) (iterations: 12)
  Depth 13
Using row 84: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65450, 3, 'NA(NA(p,p),NA(s,s))')

Current expression: ('NA', ('NA', 'q', 's'), ('NA', ('NA', 's', 'q'), ('NA', ('NA', 's', 's'), ('NA', 'Z', 'Z'))))
  Best candidate: NA(NA(q,s),NA(NA(s,q),NA(NA(s,s),NA(p,Z)))) (iterations: 7)
  Depth 14
Using row 84: (0, 0, 0, 0, 0, 0, 1, 0, 0, 65450, 3, 'NA(NA(p,p),NA(s,s))')

Current expression: ('NA', ('NA

  Best candidate: NA(NA(NA(q,NA(r,q)),NA(NA(p,r),NA(NA(Z,Z),Z))),Z) (iterations: 8)
  Depth 13
Using row 86: (0, 0, 0, 0, 0, 0, 1, 0, 0, 64512, 3, 'NA(NA(p,q),NA(p,r))')

Current expression: ('NA', ('NA', ('NA', 'q', ('NA', 'r', 'q')), ('NA', ('NA', 'p', 'r'), ('NA', ('NA', 'Z', 'Z'), 'Z'))), 'Z')
  Best candidate: NA(NA(NA(q,NA(r,q)),NA(NA(p,r),NA(NA(p,Z),Z))),Z) (iterations: 5)
  Depth 14
Using row 86: (0, 0, 0, 0, 0, 0, 1, 0, 0, 64512, 3, 'NA(NA(p,q),NA(p,r))')

Current expression: ('NA', ('NA', ('NA', 'q', ('NA', 'r', 'q')), ('NA', ('NA', 'p', 'r'), ('NA', ('NA', 'p', 'Z'), 'Z'))), 'Z')
  Best candidate: NA(NA(NA(q,NA(r,q)),NA(NA(p,r),NA(NA(p,q),Z))),Z) (iterations: 11)
  Depth 15
Using row 86: (0, 0, 0, 0, 0, 0, 1, 0, 0, 64512, 3, 'NA(NA(p,q),NA(p,r))')

Current expression: ('NA', ('NA', ('NA', 'q', ('NA', 'r', 'q')), ('NA', ('NA', 'p', 'r'), ('NA', ('NA', 'p', 'q'), 'Z'))), 'Z')
  Best candidate: NA(NA(NA(q,NA(r,q)),NA(NA(p,r),NA(NA(p,q),NA(Z,Z)))),Z) (iterations: 11)
  Depth 16


  Best candidate: NA(NA(r,NA(r,r)),NA(q,NA(q,NA(NA(p,Z),Z)))) (iterations: 3)
  Depth 13
Using row 88: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61632, 3, 'NA(NA(p,q),NA(q,r))')

Current expression: ('NA', ('NA', 'r', ('NA', 'r', 'r')), ('NA', 'q', ('NA', 'q', ('NA', ('NA', 'p', 'Z'), 'Z'))))
  Best candidate: NA(NA(r,NA(r,r)),NA(q,NA(q,NA(NA(p,NA(Z,Z)),Z)))) (iterations: 15)
  Depth 14
Using row 88: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61632, 3, 'NA(NA(p,q),NA(q,r))')

Current expression: ('NA', ('NA', 'r', ('NA', 'r', 'r')), ('NA', 'q', ('NA', 'q', ('NA', ('NA', 'p', ('NA', 'Z', 'Z')), 'Z'))))
  Best candidate: NA(NA(r,NA(r,r)),NA(q,NA(q,NA(NA(p,NA(p,Z)),Z)))) (iterations: 3)
  Depth 15
Using row 88: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61632, 3, 'NA(NA(p,q),NA(q,r))')

Current expression: ('NA', ('NA', 'r', ('NA', 'r', 'r')), ('NA', 'q', ('NA', 'q', ('NA', ('NA', 'p', ('NA', 'p', 'Z')), 'Z'))))
  Best candidate: NA(NA(r,NA(r,r)),NA(q,NA(q,NA(NA(p,NA(p,NA(Z,Z))),Z)))) (iterations: 3)
  Depth 16
Using row 88: (0, 0,

  Best candidate: NA(NA(s,p),NA(Z,Z)) (iterations: 19)
  Depth 5
Using row 94: (0, 0, 0, 0, 0, 0, 1, 0, 0, 60928, 3, 'NA(NA(p,r),NA(p,s))')

Current expression: ('NA', ('NA', 's', 'p'), ('NA', 'Z', 'Z'))
  Best candidate: NA(NA(s,p),NA(p,Z)) (iterations: 1)
  Depth 6
Using row 94: (0, 0, 0, 0, 0, 0, 1, 0, 0, 60928, 3, 'NA(NA(p,r),NA(p,s))')

Current expression: ('NA', ('NA', 's', 'p'), ('NA', 'p', 'Z'))
  Found formula: NA(NA(s,p),NA(p,r))  (0.0% longer)
Processing row 95...
  Depth 0
Using row 95: (0, 0, 0, 0, 0, 0, 1, 0, 0, 64752, 3, 'NA(NA(p,r),NA(q,q))')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 37)
  Depth 1
Using row 95: (0, 0, 0, 0, 0, 0, 1, 0, 0, 64752, 3, 'NA(NA(p,r),NA(q,q))')

Current expression: ('NA', 'Z', 'Z')
  Best candidate: NA(NA(Z,Z),Z) (iterations: 13)
  Depth 2
Using row 95: (0, 0, 0, 0, 0, 0, 1, 0, 0, 64752, 3, 'NA(NA(p,r),NA(q,q))')

Current expression: ('NA', ('NA', 'Z', 'Z'), 'Z')
  Best candidate: NA(NA(q,Z),Z) (iterations: 5)
  Depth 3
Usi

  Best candidate: NA(NA(r,Z),Z) (iterations: 15)
  Depth 3
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')

Current expression: ('NA', ('NA', 'r', 'Z'), 'Z')
  Best candidate: NA(NA(r,s),Z) (iterations: 7)
  Depth 4
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')

Current expression: ('NA', ('NA', 'r', 's'), 'Z')
  Best candidate: NA(NA(r,s),NA(Z,Z)) (iterations: 3)
  Depth 5
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')

Current expression: ('NA', ('NA', 'r', 's'), ('NA', 'Z', 'Z'))
  Best candidate: NA(NA(r,s),NA(r,Z)) (iterations: 1)
  Depth 6
Using row 98: (0, 0, 0, 0, 0, 0, 1, 0, 0, 52360, 3, 'NA(NA(p,r),NA(r,s))')

Current expression: ('NA', ('NA', 'r', 's'), ('NA', 'r', 'Z'))
  Found formula: NA(NA(r,s),NA(r,p))  (0.0% longer)
Processing row 99...
  Depth 0
Using row 99: (0, 0, 0, 0, 0, 0, 1, 0, 0, 61098, 3, 'NA(NA(p,r),NA(s,s))')

Current expression: Z
  Best candidate: NA(Z,Z) (iterations: 15)
 